In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1401,tt0432291,18000000,46201432,The Fog,Tom Welling|Maggie Grace|Selma Blair|DeRay Dav...,Rupert Wainwright,Their PAST Has Come Back To HAUNT THEM,"Trapped within an eerie mist, the residents of...",100,Horror|Thriller,Revolution Studios|Debra Hill Productions,10/14/2005,4.2,2005
93,tt2268016,14800000,122413057,Magic Mike XXL,Channing Tatum|Matt Bomer|Joe Manganiello|Kevi...,Gregory Jacobs,Back to the grind.,Three years after Mike bowed out of the stripp...,115,Comedy|Drama|Music,Warner Bros.|Iron Horse Entertainment (II),7/1/2015,6.3,2015
1060,tt1712170,45000000,30353232,Alex Cross,Tyler Perry|Rachel Nichols|Jean Reno|Matthew F...,Rob Cohen,Don't Ever Cross Alex Cross,After Washington DC detective Alex Cross is to...,101,Action|Thriller|Crime|Mystery,Summit Entertainment,10/18/2012,5.2,2012
1085,tt0242653,150000000,424988211,The Matrix Revolutions,Keanu Reeves|Laurence Fishburne|Carrie-Anne Mo...,Lilly Wachowski|Lana Wachowski,Everything that has a beginning has an end.,The human city of Zion defends itself against ...,129,Adventure|Action|Thriller|Science Fiction,Village Roadshow Pictures|NPV Entertainment|Si...,11/5/2003,6.4,2003
412,tt1193631,30000000,159291809,Step Up 3D,Rick Malambri|Adam G. Sevani|Joe Slaughter|Sha...,Jon M. Chu,Two Worlds. One Dream.,A tight-knit group of New York City street dan...,107,Drama|Romance,Summit Entertainment|Touchstone Pictures,8/4/2010,6.5,2010


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [4]:
answers = {} # словарь для ответов

data['profit'] =  data.revenue - data.budget # create a new column for Profit values (revenue-budget)
data['month'] = data.release_date.apply(lambda x: x.split('/')[0]) # create a new column for grouping by month
data['title_length'] = data.original_title.apply(lambda x: len(x)) # required for Question 24
data['list_companies'] = data.production_companies.str.split("|") # required for Question 24
data['overview_length'] = data.overview.apply(lambda x: len(x)) # required for Question 25
data.head()

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,title_length,list_companies,overview_length
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,6.5,2015,1363528810,6,14,"[Universal Studios, Amblin Entertainment, Lege...",175
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,7.1,2015,228436354,5,18,"[Village Roadshow Pictures, Kennedy Miller Pro...",581
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,6.3,2015,185238201,3,9,"[Summit Entertainment, Mandeville Films, Red W...",139
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,7.5,2015,1868178225,12,28,"[Lucasfilm, Truenorth Productions, Bad Robot]",145
4,tt2820852,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,7.3,2015,1316249360,4,9,"[Universal Pictures, Original Film, Media Righ...",91


# 1. У какого фильма из списка самый большой бюджет?

In [5]:
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'
#

In [6]:
data[data.budget==data.budget.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,title_length,list_companies,overview_length
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,Walt Disney Pictures|Jerry Bruckheimer Films|M...,5/11/2011,6.3,2011,641683000,5,43,"[Walt Disney Pictures, Jerry Bruckheimer Films...",411


# 2. Какой из фильмов самый длительный (в минутах)?

In [7]:
answers['2'] = 'Gods and Generals (tt0279111)'

In [8]:
data[data.runtime==data.runtime.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,title_length,list_companies,overview_length
1157,tt0279111,56000000,12923936,Gods and Generals,Stephen Lang|Jeff Daniels|Robert Duvall|Kevin ...,Ronald F. Maxwell,The nations heart was touched by...,The film centers mostly around the personal an...,214,Drama|History|War,Turner Pictures|Antietam Filmworks,2/21/2003,5.8,2003,-43076064,2,17,"[Turner Pictures, Antietam Filmworks]",307


# 3. Какой из фильмов самый короткий (в минутах)?





In [9]:
answers['3'] = 'Winnie the Pooh (tt1449283)'

In [10]:
data[data.runtime==data.runtime.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,title_length,list_companies,overview_length
768,tt1449283,30000000,14460000,Winnie the Pooh,Jim Cummings|Travis Oates|Jim Cummings|Bud Luc...,Stephen Anderson|Don Hall,Oh Pooh.,"During an ordinary day in Hundred Acre Wood, W...",63,Animation|Family,Walt Disney Pictures|Walt Disney Animation Stu...,4/13/2011,6.8,2011,-15540000,4,15,"[Walt Disney Pictures, Walt Disney Animation S...",305


# 4. Какова средняя длительность фильмов?


In [11]:
answers['4'] = '110'

In [12]:
data.runtime.mean()

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [13]:
answers['5'] = '107'

In [14]:
data.runtime.median()

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [15]:
answers['6'] = 'Avatar (tt0499549)'

In [16]:
data[data.profit.max()==data.profit]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,title_length,list_companies,overview_length
239,tt0499549,237000000,2781505847,Avatar,Sam Worthington|Zoe Saldana|Sigourney Weaver|S...,James Cameron,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",162,Action|Adventure|Fantasy|Science Fiction,Ingenious Film Partners|Twentieth Century Fox ...,12/10/2009,7.1,2009,2544505847,12,6,"[Ingenious Film Partners, Twentieth Century Fo...",175


# 7. Какой фильм самый убыточный? 

In [17]:
answers['7'] = 'The Lone Ranger (tt1210819)'

In [18]:
data[data.profit.min()==data.profit]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,title_length,list_companies,overview_length
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,7/3/2013,6.0,2013,-165710090,7,15,"[Walt Disney Pictures, Jerry Bruckheimer Films...",386


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [19]:
answers['8'] = '1478'

In [20]:
len(data[data.profit>0])

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [21]:
answers['9'] = 'The Dark Knight (tt0468569)'

In [22]:
data_2008 = data[data["release_year"]==2008]
data_2008[data_2008.revenue==data_2008.revenue.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,title_length,list_companies,overview_length
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,DC Comics|Legendary Pictures|Warner Bros.|Syncopy,7/16/2008,8.1,2008,816921825,7,15,"[DC Comics, Legendary Pictures, Warner Bros., ...",396


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [23]:
answers['10'] = 'The Lone Ranger (tt1210819)'

In [24]:
data_mod = data.query('2012<=release_year<=2014')
data_mod[data_mod.profit==data_mod.profit.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,title_length,list_companies,overview_length
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,7/3/2013,6.0,2013,-165710090,7,15,"[Walt Disney Pictures, Jerry Bruckheimer Films...",386


# 11. Какого жанра фильмов больше всего?

In [25]:
answers['11'] = 'Drama'

In [26]:
genre_list = []
for genre_line in data.genres:
    genre_list.extend(genre_line.split('|')) # genre_list will contain every occurrence of any genre
Counter(genre_list).most_common(1)

[('Drama', 782)]

ВАРИАНТ 2

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [27]:
answers['12'] = 'Drama'

In [28]:
data_profit_only = data[data.profit>0]
genre_list = []
for genre_line in data_profit_only.genres:
    genre_list.extend(genre_line.split('|')) # genre_list will contain every occurrence of any genre
Counter(genre_list).most_common(1)           # count the number of occurrences for each genre

[('Drama', 560)]

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [29]:
answers['13'] = 'Peter Jackson'

In [30]:
data_grouped = data.groupby(['director']).revenue.sum()

data_grouped.idxmax()

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [31]:
answers['14'] = 'Robert Rodriguez'

In [32]:
data_action = data[data.genres.str.contains('Action')]
director_list = []
for director_line in data_action.director:
    director_list.extend(director_line.split('|')) # director_list contains every occurrence of any director name
Counter(director_list).most_common(1)


[('Robert Rodriguez', 9)]

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [33]:
answers['15'] = 'Chris Hemsworth'

In [34]:
data_2012 = data.query('release_year==2012')

actors = set() # contains the set of all actor names
for actor_line in data_2012.cast:
    actors.update(actor_line.split('|'))
    
earnings = dict.fromkeys(actors, 0) # contains earning counter for each actor
for index,row in data_2012.iterrows():
    for actor in row.cast.split('|'):
        earnings[str(actor)] += row.revenue

max_key = max(earnings, key=earnings.get)
max_key
    

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [35]:
answers['16'] = 'Matt Damon'

In [36]:
data_pricey = data.query('budget>budget.mean()') # budget value has to be higher than mean budget value
actor_list = []
for actor_line in data_pricey.cast:
    actor_list.extend(actor_line.split('|')) # actor_list contains every occurrence of any actor name
Counter(actor_list).most_common(1)

[('Matt Damon', 18)]

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [37]:
answers['17'] = 'Action'

In [38]:
data_nicolas = data[data.cast.str.contains('Nicolas Cage',na=False)]
genre_list = []
for genre_line in data_nicolas.genres:
    genre_list.extend(genre_line.split('|')) # genre_list will contain every occurrence of any genre
Counter(genre_list).most_common(1)

[('Action', 17)]

# 18. Самый убыточный фильм от Paramount Pictures

In [39]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'

In [40]:
data_paramount = data[data.production_companies.str.contains('Paramount Pictures',na=False)]
data_paramount[data_paramount.profit==data_paramount.profit.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,title_length,list_companies,overview_length
925,tt0267626,100000000,35168966,K-19: The Widowmaker,Harrison Ford|Liam Neeson|Peter Sarsgaard|Joss...,Kathryn Bigelow,Fate has found its hero.,When Russia's first nuclear submarine malfunct...,138,Thriller|Drama|History,Paramount Pictures|Intermedia Films|National G...,7/19/2002,6.0,2002,-64831034,7,20,"[Paramount Pictures, Intermedia Films, Nationa...",140


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [41]:
answers['19'] = '2015'

In [42]:
df_revenue_max = data.groupby(['release_year'])['revenue'].sum()
df_revenue_max.idxmax()

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [43]:
answers['20'] = '2014'

In [44]:
df_warner = data[data.production_companies.str.contains("Warner Bros")]
df_warner.groupby(['release_year'])['profit'].sum().idxmax()


2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [45]:
answers['21'] = 'Сентябрь'

In [46]:
df_month = data.groupby(['month'])['imdb_id'].count()
df_month.idxmax()

'9'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [47]:
answers['22'] = '450'

In [48]:
df_summer = data.query('month=="8" or month=="7" or month=="6"')
len(df_summer.index)

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [49]:
answers['23'] = 'Peter Jackson'

In [50]:
df_winter = data.query('month=="12" or month=="1" or month=="2"')
director_list = []
for director_line in df_winter.director:
    director_list.extend(director_line.split('|')) # director_list contains every occurrence of any director name
Counter(director_list).most_common(1)

[('Peter Jackson', 7)]

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [51]:
answers['24'] = 'Four By Two Productions'

In [52]:
df_exploded = data.explode('list_companies') # now each movie will be repeated for each company
df_exploded.groupby(['list_companies'])['title_length'].mean().idxmax()

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [53]:
answers['25'] = 'Midnight Picture Show'

In [54]:
df_exploded = data.explode('list_companies')
df_exploded.groupby(['list_companies'])['overview_length'].mean().idxmax()

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [55]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'

In [56]:
set1 = set(data.nlargest(len(data.index)//100, 'vote_average').original_title)
set1

{'12 Years a Slave',
 'Big Hero 6',
 'Gone Girl',
 'Guardians of the Galaxy',
 'Inception',
 'Inside Out',
 'Interstellar',
 'Memento',
 'Room',
 'Spotlight',
 'The Dark Knight',
 'The Fault in Our Stars',
 'The Grand Budapest Hotel',
 'The Imitation Game',
 'The Lord of the Rings: The Return of the King',
 'The Pianist',
 'The Theory of Everything',
 'The Wolf of Wall Street'}

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [57]:
answers['27'] = 'Daniel Radcliffe & Rupert Grint'

In [58]:
actor_pair_list = []
for actor_line in data.cast:
    pairs_current_line = combinations(actor_line.split('|'), 2)
    actor_pair_list.extend(pairs_current_line)
Counter(actor_pair_list).most_common(1)

[(('Daniel Radcliffe', 'Rupert Grint'), 8)]

# Submission

In [59]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': '110',
 '5': '107',
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'Сентябрь',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [60]:
# и убедиться что ни чего не пропустил)
len(answers)

27